<a href="https://colab.research.google.com/github/jmlDC/Truth-Thesis22-23/blob/Scraping-trial/ScrapingNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install requests-html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.4 MB/s 
     |████████████████████████████████| 112 kB 27.4 MB/s 
     |████████████████████████████████| 140 kB 52.0 MB/s 
     |████████████████████████████████| 127 kB 57.7 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=139254f21e0177f8a84c46f30b5a7c04776b46e7dd2473307c048b15bb760dbd
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=ca710c278245e233b934e795900b3b05388356800a8b6fdd9b4145baf6e4f7d4
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [2]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

from requests_html import HTMLSession 



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
dir  = "/content/gdrive/MyDrive/THESIS-MS/Truth-Thesis22-23/"


In [5]:
## Politifact 
pagesToGet= 1
upperframe=[]  
site = "politifact"


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.

    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))

    
    filename= f"{dir + site}NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
data.head()

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30


,Statement,Link,Date,Source,Label
0,A royal guard member collapsed because of the ...,https://www.politifact.com/factchecks/2022/sep...,"mber 16, 2022",Instagram posts,false
1,Photo shows Queen Elizabeth II checking the oi...,https://www.politifact.com/factchecks/2022/sep...,"mber 16, 2022",Viral image,false
2,“Incomes are up.”,https://www.politifact.com/factchecks/2022/sep...,"mber 16, 2022",Joe Biden,barely-true
3,Blake Masters does not support abortion except...,https://www.politifact.com/factchecks/2022/sep...,"mber 16, 2022",Senate Majority PAC,mostly-true
4,“Mark Kelly voted against the Keystone pipelin...,https://www.politifact.com/factchecks/2022/sep...,"mber 16, 2022",Saving Arizona PAC,barely-true


In [6]:
def extract_source(url):
    agent = {"User-Agent":"Chrome/105.0.0.0"}
    try:
      source=requests.get(url, headers=agent)
    except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
    
    return source                                               #ignore this page. Abandon this and go back.



In [7]:
## Manila bulletin
pagesToGet= 1
upperframe=[]  
topic = "ukraine war"
site = "manilaBulletin"

data=pd.DataFrame(columns=['Statement','Link','Date',"Topic"])


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://mb.com.ph/page/'+str(page)+'/?s='+topic.replace(" ", "+")
    print(url)

    time.sleep(2)   
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')

    links=soup.find("article", {'class' : 'content-bar col-12 col-lg-12 order-1 col-xl order-xl-2'}).find_all('li',attrs={'class':'article'})
    print(len(links))
    
    filename= f"{dir + site + topic}-NEWS-List.csv"    
    for j in links:
        frame=[]
        try:
          Statement = j.find('h4').find('a').text
          Link = j.find('h4',attrs={'class':'title'}).find('a')['href'].strip()
          Date = j.find('time', {'class': 'time-ago'})['data-time'][0:10].strip()
          frame.extend((Statement, Link, Date, topic))
        except Exception as e:
            pass

        data = data.append(pd.Series(frame, index=data.columns[:len(frame)]), ignore_index=True)
        # upperframe.extend(frame)
data.to_csv(filename)
data.head()

processing page : 1
https://mb.com.ph/page/1/?s=ukraine+war
16


,Statement,Link,Date,Topic
0,Pope urges end to ‘senseless’ Ukraine war duri...,https://mb.com.ph/2022/09/14/pope-urges-end-to...,2022-09-14,ukraine war
1,Gatchalian seeks impact assessment of Russia-U...,https://mb.com.ph/2022/08/04/gatchalian-seeks-...,2022-08-04,ukraine war
2,"US sanctions Putin ‘girlfriend,’ more oligarch...",https://mb.com.ph/2022/08/03/us-sanctions-puti...,2022-08-03,ukraine war
3,Pacquiao-Yoo exhibition match to serve as char...,https://mb.com.ph/2022/07/20/pacquiao-yoo-exhi...,2022-07-20,ukraine war
4,Ukraine war to dominate G20 finance chief meet...,https://mb.com.ph/2022/07/15/ukraine-war-to-do...,2022-07-15,ukraine war


In [8]:
data2 = pd.DataFrame(columns=["Title", "Author", "Date", "Text"])
filename= f"{dir+site+'-'+topic}NEWS.csv"

for index, row in data.iterrows():
  url = row["Link"] 
  print(url)

  time.sleep(1)   
  soup=BeautifulSoup(extract_source(url).text, 'html.parser')

  head=soup.find("article", {'id' : 'the-article-content'})
  # print(content.prettify())
  textList =soup.find("article", {'id' : 'the-article-content'}).find("section", {'class': 'article-content'}).find_all("p")
  
  
  row=[]
  try:
    Title = head.find('h2',{'class':'title'}).text.strip()
    Author = head.find('p',attrs={'class':'author'}).find('a').text[3:].strip()
    Date = head.find('p', attrs={'class': 'published'}).text[10:-9].strip(",") 
  except Exception as e:
      pass
  
  body = ""
  for t in textList:
    body += (t.text.replace('\u200b', '')) +"\n" 

  row.extend((Title, Author, Date, body))
  

  data2 = data2.append(pd.Series(row, index=data2.columns[:len(row)]), ignore_index=True)
  
data2.to_csv(filename)
data2.head()

https://mb.com.ph/2022/09/14/pope-urges-end-to-senseless-ukraine-war-during-kazakhstan-visit/
https://mb.com.ph/2022/08/04/gatchalian-seeks-impact-assessment-of-russia-ukraine-war-on-ph/
https://mb.com.ph/2022/08/03/us-sanctions-putin-girlfriend-more-oligarchs-for-complicity-in-ukraine-war/
https://mb.com.ph/2022/07/20/pacquiao-yoo-exhibition-match-to-serve-as-charity-event-for-ukraine-war-victims/
https://mb.com.ph/2022/07/15/ukraine-war-to-dominate-g20-finance-chief-meeting-in-indonesia/
https://mb.com.ph/2022/06/27/ukraine-war-could-boost-illegal-drug-production-un/
https://mb.com.ph/2022/06/09/un-chief-warns-impact-of-ukraine-war-on-world-is-worsening/
https://mb.com.ph/2022/06/07/church-leader-calls-for-prayers-for-an-end-to-the-russia-ukraine-war/
https://mb.com.ph/2022/05/30/ph-reiterates-call-to-end-russia-ukraine-war-in-asean-russia-forum/
https://mb.com.ph/2022/05/23/ukraine-war-pandemic-push-colour-who-international-meet/
https://mb.com.ph/2022/05/10/putin-says-russia-defend

,Title,Author,Date,Text
0,Pope urges end to ‘senseless’ Ukraine war duri...,Agence-France-Presse,"September 14, 2022","NUR SULTAN, Kazakhstan – Pope Francis on Tuesd..."
1,Gatchalian seeks impact assessment of Russia-U...,Joseph Pedrajas,"August 4, 2022",Senator Sherwin Gatchalian wants an assessment...
2,"US sanctions Putin ‘girlfriend,’ more oligarch...",Agence-France-Presse,"August 3, 2022","WASHINGTON, United States – The United States ..."
3,Pacquiao-Yoo exhibition match to serve as char...,Carlo Anolin,"July 20, 2022",The showdown between Manny Pacquiao and DK Yoo...
4,Ukraine war to dominate G20 finance chief meet...,Agence-France-Presse,"July 15, 2022","BALI, Indonesia – Group of 20 finance minister..."


In [9]:
print(row)

["Legarda tells gov't to guarantee food security amid Russia-Ukraine war fallout", 'Ellson Quismorio', 'March 26, 2022', 'Former three-term Senator Loren Legarda has called on the country’s economic managers to ensure food security by helping Filipino farmers and fisherfolk.\n \n\n This, as the current House deputy speaker from Antique cited possible repercussions of a looming shortage of staple crops and rising food prices due to Russia’s invasion of Ukraine.\n Citing a report from the UN (United Nations) International Fund for Agricultural Development (IFAD), Legarda said Russia’s invasion of Ukraine last month has prevented grain shipments from the two countries which jointly account for roughly 25 percent of world wheat exports and 16 percentof corn exports. She said the crisis is now being felt in some of the world’s poorest countries, including the Philippines.\n Legarda said some countries like Turkey reportedly banned wheat flour exports to secure its own needs. Next to Ukraine

In [10]:
print(body)

Former three-term Senator Loren Legarda has called on the country’s economic managers to ensure food security by helping Filipino farmers and fisherfolk.
 

 This, as the current House deputy speaker from Antique cited possible repercussions of a looming shortage of staple crops and rising food prices due to Russia’s invasion of Ukraine.
 Citing a report from the UN (United Nations) International Fund for Agricultural Development (IFAD), Legarda said Russia’s invasion of Ukraine last month has prevented grain shipments from the two countries which jointly account for roughly 25 percent of world wheat exports and 16 percentof corn exports. She said the crisis is now being felt in some of the world’s poorest countries, including the Philippines.
 Legarda said some countries like Turkey reportedly banned wheat flour exports to secure its own needs. Next to Ukraine, she said Turkey is one of the Philippines’ biggest sources of wheat flour, the main ingredient for bread, the staple pan de s

In [26]:
## ABS CBN
pagesToGet= 1
upperframe=[]  
topic = "Queen Elizabeth Death"
site = "ABS-CBN_NEWS"

data=pd.DataFrame(columns=['Title','Link','Date',"Topic"])


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://news.abs-cbn.com/special-pages/search?q='+topic.replace(" ", "%20")+'&gsc.sort=&gsc.page='+str(pagesToGet)+'#gsc.tab=0&gsc.q='+topic.replace(" ", "%20")+'&gsc.sort=&gsc.page='+str(pagesToGet)
    print(url)

    time.sleep(2)   
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')
    links=soup.find('div', {'class':'gsc-results-wrapper-nooverlay gsc-results-wrapper-visible'}).find_all('div',{'class':'gs-webResult gs-result'})
    print((links))
    
    filename= f"{dir + site + topic}-NEWS-List.csv"    

    for j in links:
        frame=[]
        try:
          Title = j.find('div', {"clas": 'gs-title'}).find('a', {'class': 'gs-title'}).text
          print(Title)
          Link = j.find('div', {"clas": 'gs-title'}).find('a', {'class': 'gs-title'})['href'].strip()
          print(Link)
          Date = j.find('div', {'class': 'gs-bidi-start-align gs-snippet'}).text.split("<b>")[0].strip()
          print(Date)
          # frame.extend((Statement, Link, Date, topic))
        except Exception as e:
            pass

data.to_csv(filename)
data.head()

processing page : 1
https://news.abs-cbn.com/special-pages/search?q=Queen%20Elizabeth%20Death&gsc.sort=&gsc.page=1#gsc.tab=0&gsc.q=Queen%20Elizabeth%20Death&gsc.sort=&gsc.page=1
<!DOCTYPE html>

<html lang="en">
<head id="head"><title>
	Search | ABS-CBN News
</title><meta content="ie=edge,chrome=1" http-equiv="x-ua-compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/><script src="https://news.abs-cbn.com/resources/66b84909341b4a6897d5b6220b9a4e474f80016460894" type="text/javascript"></script><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-12983461dbdd35fb671","applicationID":"1528829768","transactionName":"YFFRZ0dVX0ICBxJfV1kbcmBlG1JcEBQHUV1EG0NcR0BQXRcBC0ZUVkBWHVRHQUk=","queueTime":0,"applicationTime":19432,"agent":"","atts":""}</script><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-dat

,Title,Link,Date,Topic


In [28]:
print(soup)

<!DOCTYPE html>

<html lang="en">
<head id="head"><title>
	Search | ABS-CBN News
</title><meta content="ie=edge,chrome=1" http-equiv="x-ua-compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/><script src="https://news.abs-cbn.com/resources/66b84909341b4a6897d5b6220b9a4e474f80016460894" type="text/javascript"></script><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-12983461dbdd35fb671","applicationID":"1528829768","transactionName":"YFFRZ0dVX0ICBxJfV1kbcmBlG1JcEBQHUV1EG0NcR0BQXRcBC0ZUVkBWHVRHQUk=","queueTime":0,"applicationTime":19432,"agent":"","atts":""}</script><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"1588608710",accountID:"2918328",trustKey:"2918328",xpid:"Vg8GWVVRABABUVNaDwMOUVIJ"